<a href="https://colab.research.google.com/github/JoYongJIn/YongJin-Repository/blob/main/Transfer_learning_(Image_Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install tensorflow

In [3]:
import os
import cv2
from google.colab.patches import cv2_imshow

# 이미지 폴더 경로 설정
carrot_folder = '/content/drive/MyDrive/Colab Notebooks/Images/Carrots/'
rocket_folder = '/content/drive/MyDrive/Colab Notebooks/Images/Rockets/'

# 이미지 불러오기 및 레이블링
data = []  # 데이터를 저장할 리스트
labels = []  # 레이블을 저장할 리스트

# 당근 이미지 불러오기 및 레이블링
for filename in os.listdir(carrot_folder):
    img = cv2.imread(os.path.join(carrot_folder, filename))
    if img is not None:
        data.append(img)
        labels.append(0)  # 당근을 '0'으로 레이블링

# 로켓 이미지 불러오기 및 레이블링
for filename in os.listdir(rocket_folder):
    img = cv2.imread(os.path.join(rocket_folder, filename))
    if img is not None:
        data.append(img)
        labels.append(1)  # 로켓을 '1'으로 레이블링


In [4]:
# 이미지 데이터를 같은 크기로 일치 시키고 원활히 학습시키기위해 0부터 1사이의 값으로 픽셀값을 정규화시키는 과정입니다.
resize_dim = (224, 224)  # 예를 들어, 이미지의 크기를 224x224로 설정합니다.

resized_data = []  # 크기가 조정된 이미지를 저장할 리스트
normalized_data = []  # 정규화된 이미지를 저장할 리스트

for img in data:
    # 이미지 크기 조정
    resized_img = cv2.resize(img, resize_dim, interpolation=cv2.INTER_AREA)

    # 색상 공간 변환 (예: RGB to GRAY, if needed)
    # converted_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)  # 예를 들어, 회색조로 변환

    # 이미지 정규화: 픽셀 값을 [0, 1] 범위로 조정
    normalized_img = resized_img / 255.0

    # 결과 저장
    resized_data.append(resized_img)
    normalized_data.append(normalized_img)

# 변환된 데이터를 NumPy 배열로 변환
import numpy as np
resized_data = np.array(resized_data)
normalized_data = np.array(normalized_data)


In [5]:
# 데이터셋을 Training Set, Validation Set, Test Set으로 분할 합니다.(검증데이터(파라미터를 정한다),테스트데이터셋을 각각 10%씩 할당했습니다.)
from sklearn.model_selection import train_test_split

# 전체 데이터를 훈련 데이터와 테스트 데이터로 분할 (테스트 데이터는 전체의 10%)
X_train, X_test, y_train, y_test = train_test_split(normalized_data, labels, test_size=0.1, random_state=42, stratify=labels)

# 훈련 데이터를 다시 훈련 데이터와 검증 데이터로 분할 (검증 데이터는 훈련 데이터의 11.1% - 전체의 10%를 의미)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1111, random_state=42, stratify=y_train)

# 결과 확인
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (244, 224, 224, 3)
Validation data shape: (31, 224, 224, 3)
Testing data shape: (31, 224, 224, 3)


In [6]:
num_classes = len(set(y_train))

In [7]:
import numpy as np

# y_train, y_val, y_test를 NumPy 배열로 변환(TensorFlow와 호환하기 위해서 array로 해줍니다.)
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# VGG 스타일 네트워크 정의
def build_vgg(input_shape, num_classes):
    model = models.Sequential()

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(1024, activation='relu'))  # 유닛 수를 4096에서 1024로 줄임
    model.add(layers.Dense(512, activation='relu'))   # 유닛 수를 4096에서 512로 줄임
    model.add(layers.Dense(num_classes, activation='softmax'))  # 출력 레이어의 유닛 수는 클래스 수에 맞춤

    return model

# 하이퍼파라미터 설정
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])  # 예: (224, 224, 3) 적절한 값으로 변경해야 함
num_classes = len(set(y_train))  # 클래스 개수

# 모델 생성 및 컴파일
vgg_model = build_vgg(input_shape, num_classes)
vgg_model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
# 모델 훈련 (batch_size를 32로 했더니 colab 메모리 한도를 초과하여 배치의 크기를 8로 줄여서 실행합니다.)
vgg_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# 모델 평가
vgg_model.evaluate(X_test, y_test, batch_size=32)


Epoch 1/10
8/8 [==============================] - 323s 39s/step - loss: 0.0324 - accuracy: 0.9877 - val_loss: 0.1351 - val_accuracy: 0.9355
Epoch 2/10
8/8 [==============================] - 302s 37s/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.1414 - val_accuracy: 0.9355
Epoch 3/10
8/8 [==============================] - 295s 37s/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.1890 - val_accuracy: 0.9032
Epoch 4/10
8/8 [==============================] - 312s 39s/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.1702 - val_accuracy: 0.9355
Epoch 5/10
8/8 [==============================] - 305s 38s/step - loss: 9.8987e-04 - accuracy: 1.0000 - val_loss: 0.2434 - val_accuracy: 0.9032
Epoch 6/10
8/8 [==============================] - 312s 39s/step - loss: 5.8760e-04 - accuracy: 1.0000 - val_loss: 0.1809 - val_accuracy: 0.9032
Epoch 7/10
8/8 [==============================] - 295s 37s/step - loss: 4.4015e-04 - accuracy: 1.0000 - val_loss: 0.1875 - val_accuracy: 0.9032
Epoch 8/

[0.4320564568042755, 0.9032257795333862]

In [13]:
# 학습시간이 오래걸리기 때문에 학습된 모델과 파라미터를 저장합니다.

# TensorFlow SavedModel 형식으로 전체 모델 저장
vgg_model.save('saved_model', save_format='tf')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

# 데이터를 불러오고 전처리합니다.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# ResNet Block 정의
def resnet_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=stride)(x)
        shortcut = layers.BatchNormalization()(shortcut)
    else:
        shortcut = x

    x = layers.Conv2D(filters, kernel_size, strides=stride, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(filters, kernel_size, padding="same")(x)
    x = layers.BatchNormalization()(x)

    x = layers.Add()([shortcut, x])
    x = layers.Activation("relu")(x)

    return x

# ResNet 모델 구조 만들기
def build_resnet(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(64, 7, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = resnet_block(x, 64)
    x = resnet_block(x, 128, stride=2)
    x = resnet_block(x, 256, stride=2)

    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    return keras.Model(inputs, outputs)

# 모델 생성 및 컴파일
model = build_resnet((32, 32, 3), 10)
model.compile(optimizer=keras.optimizers.Adam(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# 모델 훈련
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# 모델 평가
model.evaluate(x_test, y_test, batch_size=32)

# resnet은 학습하는데 시간이 매우 오래걸렸습니다.